In [ ]:
import numpy as np
import pandas as pd
import numba as nb
import matplotlib.pyplot as plt
import time
from pathlib import Path
from IDT_alg_VR_centred import IDTVR

In [ ]:
df_et = pd.read_csv(
    "./data/VR/Logfiles_All/GazeData/p001_gazelog_on_machine_GFK-VR_date2017.6.14_time8.49.52.csv",
    skiprows=15,
    sep=";",
    encoding="utf-8",
)

# head pose
df_et[["head_pos_x", "head_pos_y", "head_pos_z"]] = df_et[
    "head_pos"
].str.split(",", expand=True)

# gaze vector
df_et[["et_x", "et_y", "et_z"]] = df_et["bino_hitObject_pos"].str.split(
    ",", expand=True
)

# POR
df_et[["POR X", "POR Y"]] = df_et["smi_bino_por"].str.split(
    ",", expand=True
)

# fix timestamp
firstTimestamp = df_et.loc[0, "#timestamp_unity"]
df_et["elapsedTime"] = df_et["#timestamp_unity"] - firstTimestamp

print("Shape", df_et.shape)
print("Columns", df_et.columns)
df_et.head(8)

In [ ]:
df_et = df_et[
    [
        "head_pos_x",
        "head_pos_y",
        "head_pos_z",
        "et_x",
        "et_y",
        "et_z",
        "POR X",
        "POR Y",
        "elapsedTime",
        "bino_hitObject",
        "bino_hitObject_Feature",
    ]
]

# cast columns to float (except the last 3)
df_et[df_et.columns[:-3]] = df_et[df_et.columns[:-3]].astype(float)

# calculate time for seach sample
df_et["time_delta"] = df_et["elapsedTime"] - df_et["elapsedTime"].shift(1)

print("Shape", df_et.shape)
print("Columns", df_et.columns)

df_et.head(8)

In [ ]:
idt_vr = IDTVR(numba_allow=True, time_th=0.1)

t0 = time.time()
df_et_res, df_et_fixations = idt_vr.fit_compute(df_et, time="elapsedTime", debug=True)
delta_time_numba = time.time() - t0

df_et_fixations

In [ ]:
print(
    "Percentage of fixations detected",
    np.round(100 * np.mean(df_et_res["class_disp"] == 0), 2),
    "%",
)
print(
    "Percentage of saccades detected",
    np.round(100 * np.mean(df_et_res["class_disp"] == 1), 2),
    "%",
)
print("Delta time", np.round(delta_time_numba, 4), "s")


In [ ]:
df_et_fixations['duration'].describe()

In [ ]:
ivt_file = Path("C://Users/tbswe/data/et-data/VR/Logfiles_All/GazeData/p001_fixations.csv")

#fix enconding
fi = ivt_file.open('rb')
content = fi.read()
fi.close()
ivt_file_fixed = Path("C://Users/tbswe/data/et-data/VR/Logfiles_All/GazeData/p001_fixations_fixed.csv")
fo = ivt_file_fixed.open('wb')
fo.write(content.replace(b'\x00', b''))
fo.close()

df_ivt = pd.read_csv(
    ivt_file_fixed,
    sep=",",
    encoding="iso-8859-1",
)

In [ ]:
df_ivt['gazeevent_duration'].describe()